# <font color=black> Analysis - FC-based parcellation </font>
<hr style="border:1px solid black">

## <font color=black> <span style="background-color: #A0B0C0"> Imports 

In [1]:
import sys,json
import glob, os
import numpy as np
import nibabel as nib
import time

sys.path.append('/media/miplab-nas2/Data3/BMPD/hc_project/analysis/code/')

from fc_based_parcellation import FC_Parcellation

%matplotlib inline
%load_ext autoreload
%autoreload 2

## <font color=black> <span style="background-color: #A0B0C0"> Initialization

### <font color=#A0B0C0> Configuration & parameters </font>

In [2]:
with open('../../config/config_fc_based_parcellation_NK_BRtoSC.json') as config_file:
    config = json.load(config_file) # load config file
# Save 
with open('my_dict.json', 'w') as f:
    # Write the dictionary to the file in JSON format
    json.dump(my_dict, f)

### <font color=#A0B0C0> Class instantiation </font>

In [ ]:
source = 'brain'
target = 'spinalcord'
fc_parcellation = FC_Parcellation(config,source,target,'corr')

## <font color=black> <span style="background-color: #A0B0C0"> Analyses

In [ ]:
start = time.time()
# Initialize empty array for individual labels
mask_source_path = config['main_dir']+config['masks'][source]
nvox_source = np.count_nonzero(nib.load(mask_source_path).get_fdata().astype(bool))
subject_labels = np.zeros((len(fc_parcellation.config['list_subjects']),nvox_source))
for sub_id,sub in enumerate(fc_parcellation.config['list_subjects']): 
    print(f'SUBJECT {sub}')
    # Compute correlation without saving files
    dict_corr = fc_parcellation.compute_voxelwise_fc(sub=sub, 
                                                  load_from_file=True, 
                                                  save_results=True, 
                                                  mask_source_path=mask_source_path, 
                                                  mask_target_path=config['main_dir']+config['masks'][target])
    # Run clustering and save labels
    subject_labels[sub_id,:] = fc_parcellation.run_clustering(dict_corr,
                                                              k=16,algorithm='kmeans',
                                                              load_from_file=False,
                                                              save_results=True)
print("... Operation performed in %.2f s!" % (time.time() - start))

In [ ]:
group_labels, indiv_labels_relabeled = fc_parcellation.group_clustering(subject_labels)

In [ ]:
fc_parcellation.prepare_seed_map(group_labels)

In [ ]:
fc_parcellation.prepare_target_maps(indiv_labels_relabeled, load_from_file=False, save_results=True)

In [ ]:
S = np.corrcoef(dict_corr['correlations'])

In [ ]:
from sklearn.cluster import KMeans
kmeans_kwargs = {'n_clusters': 8, 'init': 'k-means++', 'max_iter': 300, 'n_init': 100}
kmeans_clusters = KMeans(**kmeans_kwargs)
kmeans_clusters.fit(S)
labels = kmeans_clusters.labels_

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(S)

In [ ]:
idx_sort = np.argsort(labels)

In [ ]:
S_reordered = S[idx_sort,:]
S_reordered = S_reordered[:,idx_sort]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(S_reordered);

In [ ]:
group_labels, indiv_labels_relabeled = fc_parcellation.group_clustering(subject_labels)

In [ ]:
indiv_labels_relabeled.shape

In [ ]:
fc_parcellation.prepare_seed_map(group_labels)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(figsize=(8, 6), dpi=80)
plt.imshow(subject_labels, interpolation='nearest', aspect='auto');

In [ ]:
figure(figsize=(8, 6), dpi=80)
plt.imshow(indiv_labels_relabeled, interpolation='nearest', aspect='auto');

In [ ]:
plt.imshow(group_labels);

In [ ]:
fc_parcellation.compute_voxelwise_correlation(load_from_file=False, save_results=False, mask_source_path=config['main_dir']+config['masks']['spinalcord'],mask_target_path=config['main_dir']+config['masks']['brain'])

In [ ]:
fc_parcellation.define_n_clusters(range(4,50))

In [ ]:
fc_parcellation.run_clustering(8,algorithm='kmeans')

In [ ]:
fc_parcellation.prepare_seed_map()

In [ ]:
fc_parcellation.prepare_target_maps()

In [ ]:
fc_parcellation.run_clustering(13,algorithm='kmeans')

In [ ]:
fc_parcellation.prepare_seed_map()

In [ ]:
fc_parcellation.prepare_target_maps()

In [ ]:
A = np.array([0,2,43,4,2,3,4,2,3])

In [ ]:
std = A.std(axis=0)
A = A/ std

In [ ]:
A.std(axis=0)